In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import time

print("🔍 INSTITUTIONAL FOOTPRINT TRACKER")
print("=" * 70)
print("\nJan 6 Winners: SIDU +28%, USAR +8%, NVTS +12%, ASTS +7%")
print("Question: Can we see THEIR positioning before the move?")
print("=" * 70)

## STEP 1: Pull Price/Volume Timeline (Dec 15 - Jan 6)
What did the days BEFORE Jan 6 look like?

In [ ]:
# Jan 6 winners
WINNERS = ['SIDU', 'USAR', 'NVTS', 'ASTS']

# Pull 4 weeks of data leading up to Jan 6
start_date = '2025-12-15'
end_date = '2026-01-06'

price_data = {}

print("\nPulling price/volume data (Dec 15 - Jan 6):")
print("-" * 70)

for ticker in WINNERS:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        if not hist.empty:
            # Add some calculated columns
            hist['volume_ma'] = hist['Volume'].rolling(window=5).mean()
            hist['volume_ratio'] = hist['Volume'] / hist['volume_ma']
            hist['daily_return'] = ((hist['Close'] - hist['Open']) / hist['Open']) * 100
            
            price_data[ticker] = hist
            print(f"✓ {ticker}: {len(hist)} trading days")
        else:
            print(f"✗ {ticker}: No data")
    except Exception as e:
        print(f"✗ {ticker}: {e}")

print(f"\nGot data for {len(price_data)} tickers")

## STEP 2: Find Accumulation Days
Days where volume was high but price didn't move much = possible accumulation

In [ ]:
print("\n" + "=" * 70)
print("ACCUMULATION SIGNAL - High volume, flat price")
print("=" * 70)

for ticker, df in price_data.items():
    print(f"\n{ticker}:")
    print("-" * 70)
    
    # Find days with high volume (>1.5x average) but small price move (<3%)
    potential_accumulation = df[
        (df['volume_ratio'] > 1.5) & 
        (abs(df['daily_return']) < 3)
    ]
    
    if len(potential_accumulation) > 0:
        print("Possible accumulation days (high vol, flat price):")
        for date, row in potential_accumulation.iterrows():
            date_str = date.strftime('%Y-%m-%d')
            print(f"  {date_str}: Vol {row['volume_ratio']:.1f}x avg, Price {row['daily_return']:+.1f}%")
    else:
        print("  No obvious accumulation days")
    
    # Show the 3 days before Jan 6
    print("\nLast 3 days before Jan 6:")
    for date, row in df.tail(4).iloc[:-1].iterrows():
        date_str = date.strftime('%Y-%m-%d')
        print(f"  {date_str}: ${row['Close']:.2f} ({row['daily_return']:+.1f}%), Vol {row['volume_ratio']:.1f}x")

## STEP 3: SEC Form 4 Check (Manual for now)
**What we need to check:** https://www.sec.gov/edgar/search/#/

For each ticker:
1. Go to SEC EDGAR
2. Search company name
3. Filter for "Form 4" (insider trades)
4. Check Dec 15 - Jan 5 for any BUY transactions (Transaction Code: P)

**I can't auto-scrape SEC easily (rate limits + parsing), but we can check manually.**

Let me show you the URLs to check:

In [ ]:
print("\n" + "=" * 70)
print("SEC FORM 4 CHECK - Manual Investigation Needed")
print("=" * 70)

for ticker in WINNERS:
    print(f"\n{ticker}:")
    print(f"  SEC Search: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={ticker}&type=4&dateb=&owner=only&count=40")
    print(f"  → Look for Transaction Code 'P' (Purchase) between Dec 15 - Jan 5")
    print(f"  → If insiders bought BEFORE Jan 6, that's a signal")

## STEP 4: News Timeline
Pull news from yfinance and map to price moves

In [ ]:
print("\n" + "=" * 70)
print("NEWS TIMELINE - When did news drop vs when did price move?")
print("=" * 70)

for ticker in WINNERS:
    print(f"\n{ticker}:")
    print("-" * 70)
    
    try:
        stock = yf.Ticker(ticker)
        news = stock.news
        
        if news:
            # Filter news from Dec 15 - Jan 6
            cutoff = datetime(2025, 12, 15)
            
            relevant = []
            for item in news[:10]:  # Check recent news
                pub_time = datetime.fromtimestamp(item.get('providerPublishTime', 0))
                if pub_time > cutoff:
                    relevant.append({
                        'date': pub_time.strftime('%Y-%m-%d %H:%M'),
                        'headline': item.get('title', 'No title')[:70]
                    })
            
            if relevant:
                for n in relevant[:5]:
                    print(f"  [{n['date']}] {n['headline']}")
            else:
                print("  No recent news found")
        else:
            print("  No news available")
    except Exception as e:
        print(f"  Error: {e}")

## STEP 5: The Verdict - Can We See THEM Moving?

For each ticker, answer:
1. **Was there accumulation?** (High vol before run)
2. **Did insiders buy?** (Form 4 check - manual)
3. **News timing?** (Before or after price moved)
4. **Could we have known?**

In [ ]:
print("\n" + "=" * 70)
print("THE VERDICT - THEIR FOOTPRINTS")
print("=" * 70)

print("""
For each Jan 6 winner, we need to answer:

1. ACCUMULATION: Did volume spike on quiet days before the run?
2. INSIDERS: Did Form 4 show buying before Jan 6? (Check SEC manually)
3. NEWS: Did news come BEFORE or AFTER the price moved?
4. PREDICTABLE: Could we have entered BEFORE the public knew?

Next step: Manually check SEC EDGAR for Form 4s.
If insiders bought Dec 20-Jan 3... that's THEIR signal we missed.

This is how we find their waltz. Not by inventing patterns.
By watching where THEY step.
""")

## HOMEWORK FOR TYR

For each winner, manually check:

**SIDU:**
- SEC EDGAR Form 4s Dec 15 - Jan 5
- Google: "SIDU $151B SHIELD contract" + exact date announced
- Did insiders know? Did they buy before?

**USAR:**
- Form 4 check
- Venezuela news timing (weekend Jan 4-5?)
- This one might be unpredictable

**NVTS:**
- Form 4 check
- Avnet/Cyient partnership announcement date
- Was it Dec news or Jan news?

**ASTS:**
- Form 4 check
- BlueBird 6 launch date (Dec 23?)
- Did price move before or after launch?

**If 3 out of 4 had insider buying or news BEFORE we saw the move → there's a waltz to learn.**